In [29]:
# importing the librtaries
import numpy as np
import nltk
import underthesea
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

In [30]:
# text from wikipedia about Elon Musk
txt = """Mặc dù là cập nhật, nâng cấp nhưng các yêu cầu quản lý theo các quy định
mới (Thông tư số 02/2018/TT-BXD ngày 06/2/2018 của Bộ Xây dựng quy định về
bảo vệ môi trường trong thi công xây dựng công trình và chế độ báo cáo công tác bảo
vệ môi trường ngành Xây dựng; Thông tư số 08/2017/TT-BXD ngày 16/5/2017 của
Bộ Xây dựng về quy định quản lý chất thải rắn xây dựng) của Bộ Xây dựng có nhiều
điểm rất khác so với phần mềm trước. Vì vậy, nhóm tác giả xây dựng phần mềm
rất mong nhận được sự đóng góp quý báu về chuyên môn và nghiệp vụ quản lý
của các cán bộ khai thác, sử dụng phần mềm để chúng tôi có cơ sở hoàn thiện, bổ
sung thêm các chức năng, tiện ích trong các phiên bản sau. """

In [31]:
def read_stopwords(filepath):
    lines = []

    # Open the file for reading
    with open(filepath, 'r', encoding='utf-8') as file:
        # Read each line in the file
        for line in file:
            # Append each line to the list, removing leading and trailing whitespace
            lines.append(line.strip())
            
    return lines

In [32]:
vi_stopwords = read_stopwords("./vietnamese-stopwords.txt")

In [33]:
#class for preprocessing and creating word embedding
class Preprocessing:
    #constructor
    def __init__(self,txt):
        # Tokenization
        nltk.download('punkt')  #punkt is nltk tokenizer 
        # breaking text to sentences
        tokens = underthesea.sent_tokenize(txt) 
        self.tokens = tokens
        self.tfidfvectoriser=TfidfVectorizer()

    # Data Cleaning
    # remove extra spaces
    # convert sentences to lower case 
    # remove stopword
    def clean_sentence(self, sentence, stopwords=False):
        sentence = sentence.lower().strip()
        sentence = re.sub(r'[^a-zA-Z0-9\sÀÁẢẠÃĂẰẮẲẶẴÂẦẤẨẬẪÈÉẺẸẼÊỀẾỂỆỄÌÍỈỊĨÒÓỎỌÕÔỒỐỔỘỖƠỜỚỞỢỠÙÚỦỤŨƯỪỨỬỰỮỲÝỶỴỸàáảạãăằắẳặẵâầấẩậẫèéẻẹẽêềếểệễìíỉịĩòóỏọõôồốổộỗơờớởợỡùúủụũưừứửựữỳýỷỵỹđ]+', '', sentence)
        if stopwords:
            sentence = remove_stopwords(sentence)
        return sentence

    # store cleaned sentences to cleaned_sentences
    def get_cleaned_sentences(self,tokens, stopwords=False):
        cleaned_sentences = []
        for line in tokens:
            cleaned = self.clean_sentence(line, stopwords)
            cleaned_sentences.append(cleaned)
        return cleaned_sentences

    #do all the cleaning
    def cleanall(self):
        cleaned_sentences = self.get_cleaned_sentences(self.tokens, stopwords=True)
        cleaned_sentences_with_stopwords = self.get_cleaned_sentences(self.tokens, stopwords=False)
        # print(cleaned_sentences)
        # print(cleaned_sentences_with_stopwords)
        return [cleaned_sentences,cleaned_sentences_with_stopwords]

    # TF-IDF Vectorizer
    def TFIDF(self,cleaned_sentences):
        self.tfidfvectoriser.fit(cleaned_sentences)
        tfidf_vectors=self.tfidfvectoriser.transform(cleaned_sentences)
        return tfidf_vectors

    #tfidf for question
    def TFIDF_Q(self,question_to_be_cleaned):
        tfidf_vectors=self.tfidfvectoriser.transform([question_to_be_cleaned])
        return tfidf_vectors

    # main call function
    def doall(self):
        cleaned_sentences, cleaned_sentences_with_stopwords = self.cleanall()
        tfidf = self.TFIDF(cleaned_sentences)
        return [cleaned_sentences,cleaned_sentences_with_stopwords,tfidf]


In [34]:
#class for answering the question.
class AnswerMe:
    #cosine similarity
    def Cosine(self, question_vector, sentence_vector):
        dot_product = np.dot(question_vector, sentence_vector.T)
        denominator = (np.linalg.norm(question_vector) * np.linalg.norm(sentence_vector))
        return dot_product/denominator
    
    #Euclidean distance
    def Euclidean(self, question_vector, sentence_vector):
        vec1 = question_vector.copy()
        vec2 = sentence_vector.copy()
        if len(vec1)<len(vec2): vec1,vec2 = vec2,vec1
        vec2 = np.resize(vec2,(vec1.shape[0],vec1.shape[1]))
        return np.linalg.norm(vec1-vec2)

    # main call function
    def answer(self, question_vector, sentence_vector, method):
        if method==1: return self.Euclidean(question_vector,sentence_vector)
        else: return self.Cosine(question_vector,sentence_vector)


In [35]:
def RetrieveAnswer(question_embedding, tfidf_vectors,method=1):
    similarity_heap = []
    if method==1: max_similarity = float('inf')
    else: max_similarity = -1
    index_similarity = -1

    for index, embedding in enumerate(tfidf_vectors):  
        find_similarity = AnswerMe()
        similarity = find_similarity.answer((question_embedding).toarray(),(embedding).toarray() , method).mean()
        if method==1:
            heapq.heappush(similarity_heap,(similarity,index))
        else:
            heapq.heappush(similarity_heap,(-similarity,index))
            
    return similarity_heap


In [36]:
# Put Your question here
user_question = "Tại sao nhóm tác giả xây dựng phần mềm này"
#define method
method = 1

In [37]:
preprocess = Preprocessing(txt)
cleaned_sentences,cleaned_sentences_with_stopwords,tfidf_vectors = preprocess.doall()

question = preprocess.clean_sentence(user_question, stopwords=True)
question_embedding = preprocess.TFIDF_Q(question)

similarity_heap = RetrieveAnswer(question_embedding , tfidf_vectors ,method)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hoang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
print("Question: ", user_question)

# number of relevant solutions you want here it will print 2
number_of_sentences_to_print = 2

while number_of_sentences_to_print>0 and len(similarity_heap)>0:
    x = similarity_heap.pop(0)
    print(cleaned_sentences_with_stopwords[x[1]])
    number_of_sentences_to_print-=1

Question:  Tại sao nhóm tác giả xây dựng phần mềm này
vì vậy nhóm tác giả xây dựng phần mềm
rất mong nhận được sự đóng góp quý báu về chuyên môn và nghiệp vụ quản lý
của các cán bộ khai thác sử dụng phần mềm để chúng tôi có cơ sở hoàn thiện bổ
sung thêm các chức năng tiện ích trong các phiên bản sau
mặc dù là cập nhật nâng cấp nhưng các yêu cầu quản lý theo các quy định
mới thông tư số 022018ttbxd ngày 0622018 của bộ xây dựng quy định về
bảo vệ môi trường trong thi công xây dựng công trình và chế độ báo cáo công tác bảo
vệ môi trường ngành xây dựng thông tư số 082017ttbxd ngày 1652017 của
bộ xây dựng về quy định quản lý chất thải rắn xây dựng của bộ xây dựng có nhiều
điểm rất khác so với phần mềm trước
